In [1]:
import sys
sys.path.append('../src')
from transformer_encoder import TransformerEncoderV3  
from positional_encoding import encode_pos_sin_cosine
import seaborn as sns
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

2024-05-12 12:56:20.766603: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 12:56:20.795242: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 12:56:21.548388: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-12 12:56:24.213741: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 12:56:31.456696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
# from datasets import load_dataset

# # Load an example dataset, 'wikipedia' for English, 2020-03-01 version
# dataset = load_dataset("wikipedia", "20220301.en", split=["train"])
# # print(dataset[0])
# article_texts_dataset = dataset[0]['text'][:1000]
# # Extract text and write to a file
# with open('input_text.txt', 'w', encoding='utf-8') as f:
#     for article  in article_texts_dataset:
#         # Write each Wikipedia article on a new line
#         f.write(article.replace('\n', ' ') + '\n')

In [3]:
# print(dataset[0]['text'][1])

In [4]:
# !python create_pretraining_data.py --vocab_file vocab.txt --input_text input_text.txt --output_tfrecord output.tfrecord --do_lower_case --nsp


In [5]:
vocab_size = 30000  # Smaller vocabulary size for simplicity
num_layers = 2  # Fewer layers
d_model = 256  # Smaller dimensionality
num_heads = 4
dff = 512
segment_size = 2
max_seq_length = 128

In [ ]:
import tensorflow as tf

def load_and_print_dataset(filepath):
    raw_dataset = tf.data.TFRecordDataset(filepath)
    for i, raw_record in enumerate(raw_dataset):  # Adjust the number based on how many you want to check
        # print("Raw record:", raw_record.numpy())
        try:
            example = tf.io.parse_single_example(
                raw_record,
                {
                    'input_ids': tf.io.FixedLenFeature([128], tf.int64),
                    'segment_ids': tf.io.FixedLenFeature([128], tf.int64),
                    'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
                    'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
                    'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
                }
            )
            # print("Parsed example:", example)
        except Exception as e:
            print(f"Failed to parse record {i}: {e}")
            # break  # Or continue based on how you want to handle errors

# Example usage
load_and_print_dataset('output.tfrecord')


Failed to parse record 180: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 256: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 300: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 308: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica

2024-05-12 17:00:18.412166: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.412220: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.452865: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.452916: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.473710: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.473757: W tens

Failed to parse record 597: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 600: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 632: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 640: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica

2024-05-12 17:00:18.620202: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.620249: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.622373: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.622408: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.638670: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:18.638718: W tens

Failed to parse record 994: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1002: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1004: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1010: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/repl

...

Failed to parse record 1382: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1387: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1397: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1400: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/rep

2024-05-12 17:00:19.029507: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.029546: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.032747: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.032776: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.037693: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.037757: W tens

Failed to parse record 1735: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1736: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1741: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 1742: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/rep

2024-05-12 17:00:19.231522: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.231558: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.233529: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.233563: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.236475: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 17:00:19.236513: W tens

Failed to parse record 2121: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 2122: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 2123: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example. [Op:ParseExampleV2]
Failed to parse record 2126: {{function_node __wrapped__ParseExampleV2_Tdense_3_num_sparse_2_ragged_split_types_0_ragged_value_types_0_sparse_types_2_device_/job:localhost/rep

In [26]:
def parse_tfrecord(serialized_example):
    feature_description = {
        'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
        'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
        'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
        'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
        'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
    }
    print(serialized_example)
    example = tf.io.parse_single_example(serialized_example, feature_description)
    print(example)
    input_ids = tf.cast(example['input_ids'], tf.int32)
    segment_ids = tf.cast(example['segment_ids'], tf.int32)
    masked_lm_positions = tf.sparse.to_dense(example['masked_lm_positions'])
    masked_lm_labels = tf.sparse.to_dense(example['masked_lm_labels'])
    next_sentence_labels = tf.cast(example['next_sentence_labels'], tf.int32)
    inputs = {'input_ids': input_ids, 'segment_ids': segment_ids}
    labels = {'masked_lm_positions': masked_lm_positions,
              'mlm_labels': masked_lm_labels, 'nsp_labels': next_sentence_labels}
    return (inputs, labels)

def load_dataset(filepath, batch_size):
    raw_dataset = tf.data.TFRecordDataset(filepath)
    parsed_dataset = raw_dataset.map(parse_tfrecord)
    # Ensure both inputs and labels are not None (conservative check)
    # filtered_dataset = parsed_dataset.filter(lambda x: x[0] is not None and x[1] is not None)
    batched_dataset = parsed_dataset.batch(batch_size)
    return batched_dataset

# Usage
batch_size = 32
train_dataset = load_dataset('output.tfrecord', batch_size)

try:
    for idx, (inputs, labels) in enumerate(train_dataset):
        print(f"instance {idx + 1}:")
        # print(f"Inputs: {inputs}")
        # print(f"Labels: {labels}")
        if idx == 10:  # Limit to 2 batches for demonstration
            break
except Exception as e:
    print(f"Error while processing the dataset: {e}")

{'masked_lm_labels': SparseTensor(indices=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:0", shape=(None, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:2", shape=(None,), dtype=int64), dense_shape=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:4", shape=(1,), dtype=int64)), 'masked_lm_positions': SparseTensor(indices=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:1", shape=(None, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(None,), dtype=int64), dense_shape=Tensor("ParseSingleExample/ParseExample/ParseExampleV2:5", shape=(1,), dtype=int64)), 'input_ids': <tf.Tensor 'ParseSingleExample/ParseExample/ParseExampleV2:6' shape=(128,) dtype=int64>, 'next_sentence_labels': <tf.Tensor 'ParseSingleExample/ParseExample/ParseExampleV2:7' shape=() dtype=int64>, 'segment_ids': <tf.Tensor 'ParseSingleExample/ParseExample/ParseExampleV2:8' shape=(128,) dtype=int64>}
instance 1:
instance 2:
instance 

2024-05-12 16:03:29.700327: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 16:03:29.700546: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]


In [13]:
def test_dataset(filepath):
    # dataset = load_dataset(filepath, batch_size=16)  # Using batch size of 1 for simplicity
    try:
        for i, (inputs, labels) in enumerate(train_dataset):        
            print(f"Successfully parsed entry {i+1}")        
            if i == 100:  # Optionally limit the number of entries to check
                break
    except Exception as e:
            print(f'record: {i}  , Err: {e}')

test_dataset('output.tfrecord')

Successfully parsed entry 1
Successfully parsed entry 2
Successfully parsed entry 3
Successfully parsed entry 4
Successfully parsed entry 5
Successfully parsed entry 6
Successfully parsed entry 7
Successfully parsed entry 8
Successfully parsed entry 9
Successfully parsed entry 10
Successfully parsed entry 11
record: 10  , Err: {{function_node __wrapped__IteratorGetNext_output_types_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 


2024-05-12 12:57:48.596629: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
2024-05-12 12:57:48.596902: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]


In [ ]:
single_test_instance = iter(train_dataset.take(1)).next()
single_input_tuple = single_test_instance[0]['input_ids'], single_test_instance[0]['segment_ids']
print(single_test_instance)
print()

In [8]:
class PositionalAndSegmentEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, segment_size, d_model, max_pos=2048, pos_dropout=0.1, **kwargs):
        super().__init__(**kwargs)  # Initialize the superclass (Layer)
        self.d_model = d_model  # Store the dimensionality of the model embeddings
        self.token_embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.segment_embedding = tf.keras.layers.Embedding(segment_size, d_model)
        self.pos_encoding = encode_pos_sin_cosine(max_pos, d_model, debug=False)
        self.dropout = tf.keras.layers.Dropout(pos_dropout)

    def compute_mask(self, inputs, *args, **kwargs):
        # Assuming the input structure is a tuple of (tokens, segments)
        token_inputs, _ = inputs
        return self.token_embedding.compute_mask(token_inputs, *args, **kwargs)

    def call(self, inputs, training=False):
        # Expect inputs to be a tuple (token_inputs, segment_inputs)
        token_inputs, segment_inputs = inputs
        tokens = self.token_embedding(token_inputs)  # Token embeddings
        segments = self.segment_embedding(segment_inputs)  # Segment embeddings       
        x = tokens + segments
        # Scale the embeddings by the square root of the embedding dimension size
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # Add positional encoding to the combined embeddings, sliced to match the input length
        length = tf.shape(x)[1]
        # pos_encodings = tf.reshape(self.pos_encoding, (1, -1, self.d_model))[:, :length, :]
        pos_encodings = tf.cast(tf.reshape(self.pos_encoding, (1, -1, self.d_model))[:, :tf.shape(x)[1], :], tf.float32)
        x += pos_encodings
        x = self.dropout(x, training=training)
        return x

embedding_layer = PositionalAndSegmentEmbedding(vocab_size=10000, segment_size=2, d_model=256)

# Extract a single batch from the dataset
for inputs, labels in train_dataset.take(1):
    # The inputs dictionary contains 'input_ids' and 'segment_ids'
    input_ids = inputs['input_ids']
    segment_ids = inputs['segment_ids']

    # Call the embedding layer
    embeddings = embedding_layer((input_ids, segment_ids))

    # Print the output shape
    print("Output shape:", embeddings)
    print("Output shape:", embeddings._keras_mask)
    break

Output shape: tf.Tensor(
[[[ 1.2772640e+00 -2.4091399e-01  8.5570049e-01 ... -3.6876512e-01
    7.3163813e-01  6.9305360e-02]
  [ 9.0144825e-01  3.2527626e-01  1.1216953e+00 ... -6.5900987e-01
    8.7406379e-01  3.3931565e-01]
  [ 1.2585175e+00  2.0384485e-01  1.9434187e+00 ... -1.1339396e+00
    6.0389382e-01  9.3163204e-01]
  ...
  [ 3.6662638e-01 -1.8983996e+00  5.9676629e-01 ...  1.6930604e-01
   -7.0370570e-02  4.9811387e-01]
  [ 9.4644606e-01 -2.3224136e-01  6.4128637e-04 ...  1.6334641e-01
   -4.9696949e-01 -8.2276577e-01]
  [ 2.4783790e+00 -2.0494503e-01 -4.0099782e-01 ... -1.9866620e-01
   -7.7257955e-01  3.7066635e-01]]], shape=(1, 128, 256), dtype=float32)
Output shape: tf.Tensor(
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   

In [9]:
class TransformerEncoderV4(TransformerEncoderV3):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_pos=2048, pos_dropout=0.1, **kwargs):
        super(TransformerEncoderV4, self).__init__(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, vocab_size=vocab_size, max_pos=max_pos, **kwargs)
        # Use the custom embedding layer that handles tokens, segments, and positional encodings
        self.embedding_layer = PositionalAndSegmentEmbedding(vocab_size, segment_size, d_model, max_pos, pos_dropout)

    def call(self, inputs, training=False):
        input_ids, segment_ids = inputs
        # The embedding layer now handles everything including token, segment, and positional embeddings
        x = self.embedding_layer((input_ids, segment_ids), training=training)
        x = self.enc_layers_0(x, training=training)
        for i in range(self.remaining_layers):
            x = self.enc_layers[i](x, training=training)
        return x
tren = TransformerEncoderV4(num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_pos=max_seq_length)

encoder_out = tren(single_input_tuple)
print(encoder_out)

tf.Tensor(
[[[-2.0559862   0.68244684  0.8406538  ... -0.30228087  1.2814008
   -0.7477844 ]
  [-0.07390715  2.4449873   2.0039835  ...  0.14209713 -0.04775164
   -1.9882263 ]
  [ 0.56867564  1.7794493   1.3373998  ... -0.0092607   0.5339716
   -1.5259957 ]
  ...
  [-0.32060108 -1.3991967   1.210192   ... -0.2921177   0.83640355
    0.689962  ]
  [-0.6356008  -0.73561066  0.73133075 ...  0.16209851  1.5381902
   -0.64576745]
  [ 0.62640816  0.73777807  0.36063844 ... -0.5101741   0.53488815
   -0.3078198 ]]], shape=(1, 128, 256), dtype=float32)


In [10]:
class BERT(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_seq_length=128, rate=0.1):
        super(BERT, self).__init__()
        self.encoder = TransformerEncoderV4(num_layers=num_layers, d_model=d_model, num_heads=num_heads,
                                            dff=dff, vocab_size=vocab_size, segment_size=segment_size,
                                            max_pos=max_seq_length, pos_dropout=rate)
        self.mlm_dense = tf.keras.layers.Dense(vocab_size, activation='softmax')  # Ensures output shape is [batch, seq_length, vocab_size]
        self.nsp_dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.encoder((inputs['input_ids'], inputs['segment_ids']), training=training)
        mlm_output = self.mlm_dense(x)  # Check shapes here
        nsp_output = self.nsp_dense(x[:, 0, :])
        return {'mlm_output': mlm_output, 'nsp_output': nsp_output}


bert_model = BERT(num_layers, d_model, num_heads, 
                  dff, vocab_size, segment_size)
bert_out = bert_model(single_test_instance[0])
print(bert_out)

{'mlm_output': <tf.Tensor: shape=(1, 128, 30000), dtype=float32, numpy=
array([[[4.1931282e-05, 3.3307286e-05, 3.5870740e-05, ...,
         3.2418735e-05, 3.3917240e-05, 3.2401622e-05],
        [4.5056226e-05, 3.3492379e-05, 3.7764330e-05, ...,
         3.1899155e-05, 3.8886381e-05, 2.9534836e-05],
        [3.9399289e-05, 2.9623558e-05, 3.1704592e-05, ...,
         3.0818359e-05, 3.5406229e-05, 3.0465213e-05],
        ...,
        [3.7818507e-05, 2.9007975e-05, 3.3100416e-05, ...,
         2.6583950e-05, 2.9160223e-05, 3.0038873e-05],
        [2.8480588e-05, 3.4055389e-05, 2.9500383e-05, ...,
         2.4580471e-05, 2.9309274e-05, 2.8358580e-05],
        [3.7237533e-05, 3.2789056e-05, 3.2988140e-05, ...,
         2.5015424e-05, 2.8515075e-05, 3.3566601e-05]]], dtype=float32)>, 'nsp_output': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.37367406]], dtype=float32)>}


In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_object_mlm = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                                reduction=tf.keras.losses.Reduction.NONE)
loss_object_nsp = tf.keras.losses.BinaryCrossentropy(from_logits=True)


def compute_mlm_loss(masked_positions, masked_labels, logits):
    # Gather the logits at the masked positions
    masked_logits = tf.gather(logits, masked_positions, batch_dims=1, axis=1)
    
    # Ensure that the masked_labels used here are the correct length and match the number of masked_positions
    mlm_loss = tf.keras.losses.sparse_categorical_crossentropy(masked_labels, masked_logits, from_logits=True)

    # Reduce mean across batches if needed or sum as appropriate
    return tf.reduce_mean(mlm_loss)


@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = bert_model(inputs, training=True)  # Predictions will have 'mlm_output' and 'nsp_output'
        # Compute the MLM loss using the positions and labels
        loss_mlm = compute_mlm_loss(labels['masked_lm_positions'], 
                                    labels['mlm_labels'], predictions['mlm_output'])
        # NSP loss remains the same
        loss_nsp = loss_object_nsp(labels['nsp_labels'], predictions['nsp_output'])
        total_loss = loss_mlm + loss_nsp

    gradients = tape.gradient(total_loss, bert_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, bert_model.trainable_variables))
    return total_loss, loss_mlm, loss_nsp

# Ensure that your dataset loading function is correctly parsing and returning 'masked_lm_positions' and 'masked_lm_labels'

batch_size = 16
train_dataset = load_dataset('output.tfrecord', batch_size)
# Training loop
epochs = 3
for epoch in range(epochs):
    step = 0
    for inputs, labels in train_dataset:
        loss_values = train_step(inputs, labels)
        if step % 10 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Total Loss: {loss_values[0].numpy():.4f}, MLM Loss: {loss_values[1].numpy():.4f}, NSP Loss: {loss_values[2].numpy():.4f}")
        step += 1

Epoch 1, Step 0, Total Loss: 11.0812, MLM Loss: 10.3090, NSP Loss: 0.7722


2024-05-12 12:57:05.221804: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.


Epoch 1, Step 10, Total Loss: 11.0628, MLM Loss: 10.3090, NSP Loss: 0.7539


2024-05-12 12:57:06.807295: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Key: segment_ids.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: segment_ids.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

In [ ]:
# class Human:
#     def __init__(self, name, legs=2):
#         self.name = name
#         self.legs = legs

# class MaleHuman(Human):
#     def __init__(self, *args, **kwargs):
#         super(MaleHuman, self).__init__(*args, **kwargs)
#         # self.beard = beard

# m1 = MaleHuman('Bhujay')
# m1.name

In [21]:
# def parse_tfrecord(serialized_example):
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
#         'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
#     }
#     try:
#         example = tf.io.parse_single_example(serialized_example, feature_description)    
#         input_ids = tf.cast(example['input_ids'], tf.int32)
#         segment_ids = tf.cast(example['segment_ids'], tf.int32)
#         # segment_ids = tf.sparse.to_dense(segment_ids, default_value=0)  # Default to 0
#         # segment_ids = tf.reshape(segment_ids, [max_seq_length]) 
#         masked_lm_positions = tf.sparse.to_dense(example['masked_lm_positions'])
#         masked_lm_labels = tf.sparse.to_dense(example['masked_lm_labels'])   
#         next_sentence_labels = tf.cast(example['next_sentence_labels'], tf.int32)
#         inputs = {'input_ids': input_ids, 'segment_ids': segment_ids, }
#         labels = {'masked_lm_positions': masked_lm_positions, 
#                   'mlm_labels': masked_lm_labels, 'nsp_labels': next_sentence_labels}
#     except Exception as e:
#         print(f"Failed to parse example: {e}")
#         # Return dummy/default data to allow pipeline to continue
#         inputs = {
#             'input_ids': tf.constant([0]*args.max_seq_length, dtype=tf.int32),
#             'segment_ids': tf.constant([0]*args.max_seq_length, dtype=tf.int32)
#             }
#         labels = {
#             'masked_lm_positions': tf.constant([-1]*20, dtype=tf.int32),  # Adjust 20 to your max_predictions_per_seq
#             'masked_lm_labels': tf.constant([-1]*20, dtype=tf.int32),
#             'nsp_labels': tf.constant([0], dtype=tf.int32)
#             } 
#         return (None, None)
#     return inputs, labels




# def load_dataset(filepath, batch_size):
#     raw_dataset = tf.data.TFRecordDataset(filepath)
#     parsed_dataset = raw_dataset.map(parse_tfrecord)
#     filtered_dataset = parsed_dataset.filter(lambda x: x[0] is not None and x[1] is not None)
#     batched_dataset = filtered_dataset.batch(batch_size)
#     return batched_dataset
# # Usage
# batch_size = 16
# train_dataset = load_dataset('output.tfrecord', batch_size)

# single_test_instance = iter(train_dataset.take(1)).next()
# single_input_tuple = single_test_instance[0]['input_ids'], single_test_instance[0]['segment_ids']
# print(single_test_instance)
# print()
# # print(single_input_tuple)

In [ ]:
# def _parse_function(proto):
#     # Define your tfrecord again. It must be the same as the one used for saving your data.
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([128], tf.int64),  # Assuming input_ids are of length 128
#         'segment_ids': tf.io.FixedLenFeature([128], tf.int64),  # Assuming segment_ids are of length 128
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64),
#     }

#     # Load one example
#     parsed_features = tf.io.parse_single_example(proto, feature_description)
    
#     # Turn your sparse array into a dense array with default values as 0
#     parsed_features['masked_lm_positions'] = tf.sparse.to_dense(parsed_features['masked_lm_positions'], default_value=0)
#     parsed_features['masked_lm_labels'] = tf.sparse.to_dense(parsed_features['masked_lm_labels'], default_value=0)

#     return parsed_features
# # Read the TFRecord file
# def load_dataset(file_path):
#     dataset = tf.data.TFRecordDataset(file_path)
#     dataset = dataset.map(_parse_function)  # Parse the record into tensors.
#     return dataset

# # Path to the TFRecord file
# tfrecord_file_path = 'output.tfrecord'

# # Load the dataset
# parsed_dataset = load_dataset(tfrecord_file_path)
# # Display a few examples from the dataset
# for parsed_record in parsed_dataset.take(2):  # Only take first 5 examples
#     print('Input IDs:', parsed_record['input_ids'].numpy())
#     print('Segment IDs:', parsed_record['segment_ids'].numpy())
#     print('Masked LM Positions:', parsed_record['masked_lm_positions'].numpy())
#     print('Masked LM Labels:', parsed_record['masked_lm_labels'].numpy())
#     print('Next Sentence Label:', parsed_record['next_sentence_labels'].numpy())
#     print('---')


In [ ]:
# def parse_tfrecord(serialized_example):
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([128], tf.int64),
#         'segment_ids': tf.io.FixedLenFeature([128], tf.int64),
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
#     }
#     example = tf.io.parse_single_example(serialized_example, feature_description)

#     input_ids = tf.cast(example['input_ids'], tf.int32)
#     segment_ids = tf.cast(example['segment_ids'], tf.int32)
#     masked_lm_positions = tf.sparse.to_dense(example['masked_lm_positions'])
#     masked_lm_labels = tf.sparse.to_dense(example['masked_lm_labels'])
#     next_sentence_labels = tf.cast(example['next_sentence_labels'], tf.int32)

#     inputs = {'input_ids': input_ids, 'segment_ids': segment_ids}
#     labels = {'mlm_output': masked_lm_labels, 'nsp_output': next_sentence_labels}

#     return inputs, labels

In [ ]:
# optimizer = Adam(learning_rate=2e-5)
# loss = {
#     'mlm_output': SparseCategoricalCrossentropy(from_logits=True),
#     'nsp_output': BinaryCrossentropy(from_logits=True)
# }
# metrics = {
#     'mlm_output': 'accuracy',
#     'nsp_output': 'accuracy'
# }



# def masked_sparse_categorical_crossentropy(y_true, y_pred):
#     # Create a mask to ignore `-1` in labels for loss calculation
#     mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)
#     # Compute sparse categorical crossentropy loss
#     loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
#     # Apply the mask
#     loss *= mask
#     # Calculate mean loss only over non-masked elements
#     return tf.reduce_sum(loss) / tf.reduce_sum(mask)

# # Ensure the custom loss function is used when compiling the model
# bert_model.compile(
#     optimizer=optimizer,
#     loss={'mlm_output': masked_sparse_categorical_crossentropy, 'nsp_output': BinaryCrossentropy(from_logits=True)},
#     metrics=metrics
# )
# # epochs = 3 
# # bert_model.fit(train_dataset, epochs=epochs)

In [ ]:
# class SmallBERT(Model):
#     def __init__(self, vocab_size, num_layers, d_model, num_heads, dff, max_pos=512, rate=0.1):
#         super(SmallBERT, self).__init__()
#         self.token_embedding = Embedding(vocab_size, d_model)
#         self.position_embedding = Embedding(max_pos, d_model)
#         self.segment_embedding = Embedding(2, d_model)  # Only 2 segments assumed
        
#         self.enc_layers = [TransformerEncoderV3(num_layers=1, d_model=d_model, num_heads=num_heads, dff=dff,
#                                                 vocab_size=vocab_size, max_pos=max_pos) for _ in range(num_layers)]
        
#         self.dropout = Dropout(rate)
#         self.final_layer = Dense(vocab_size)  # Prediction layer for MLM
#         self.nsp_classifier = Dense(2, activation='softmax')  # NSP output

#     def call(self, input_ids, segment_ids, training=False):
#         seq_length = tf.shape(input_ids)[1]
#         position_ids = tf.range(seq_length)
        
#         x = self.token_embedding(input_ids) + self.position_embedding(position_ids) + self.segment_embedding(segment_ids)
#         x = self.dropout(x, training=training)
        
#         for encoder in self.enc_layers:
#             x = encoder(x)
        
#         logits = self.final_layer(x)
#         pooled_output = self.dropout(sequence_output[:, 0, :], training=training)  # Use the output of the [CLS] token
#         nsp_output = self.nsp_classifier(pooled_output)
#         return logits, nsp_output  

# vocab_size = 10000  # Smaller vocabulary size for simplicity
# num_layers = 2  # Fewer layers
# d_model = 128  # Smaller dimensionality
# num_heads = 4
# dff = 512
# small_bert = SmallBERT(vocab_size, num_layers, d_model, num_heads, dff)